### TP2_ALG2         Bruno Martins 

In [ ]:

import numpy as np
import pandas as pd
import networkx as nx
from scipy.spatial import distance_matrix
from scipy.spatial.distance import euclidean
from networkx.algorithms.matching import max_weight_matching
from networkx.algorithms.euler import eulerian_circuit



In [ ]:

def read_tsp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    
    node_coord_start = lines.index("NODE_COORD_SECTION\n") + 1
    node_coord_end = lines.index("EOF\n")

    
    node_coords = [list(map(float, line.split()[1:])) for line in lines[node_coord_start:node_coord_end]]

    return node_coords

### Branch_and_Bound

Atenção: Esse primeiro algoritmo não foi desenvolvido por mim, é um código público encontrado na internet. Utilizei esse algoritmo para comparar com o meu para ver se rodava em menos tempo. Entretanto, mesmo com esse algoritmo que teoricamente é melhor implementado, não consegui rodar para nenhuma instância. Esse primeiro algoritmo é fortemente baseado no geeks for geeks.

In [ ]:
import numpy as np



def min_1(distance_matrix, i): 
    vector = distance_matrix[i,:].tolist()
    idx    = np.argsort(vector)
    m1     = vector[idx[1]]
    return m1


def min_2(distance_matrix, i): 
    vector = distance_matrix[i,:].tolist()
    idx    = np.argsort(vector)
    m2     = vector[idx[2]]
    return m2 




def explore_path(route, distance, distance_matrix, bound, weight, level, path, visited):  
    if (level == distance_matrix.shape[0]): 
        if (distance_matrix[path[level - 1], path[0]] != 0): 
            dist = weight + distance_matrix[path[level - 1], path[0]] 
            if (dist < distance): 
                distance                             = dist 
                route[:distance_matrix.shape[0] + 1] = path[:]
                route[distance_matrix.shape[0]]      = path[0]
        return route, distance, bound, weight, path, visited
    for i in range(0, distance_matrix.shape[0]): 
        if (distance_matrix[path[level-1], i] != 0 and visited[i] == False): 
            temp   = bound 
            weight = weight + distance_matrix[path[level - 1], i] 
            if (level == 1): 
                bound = bound - ((min_1(distance_matrix, path[level - 1]) + min_1(distance_matrix, i)) / 2) 
            else: 
                bound = bound - ((min_2(distance_matrix, path[level - 1]) + min_1(distance_matrix, i)) / 2)  
            if (bound + weight < distance): 
                path[level] = i 
                visited[i]  = True
                route, distance, bound, weight, path, visited = explore_path(route, distance, distance_matrix, bound, weight, level + 1, path, visited) 
            weight  = weight - distance_matrix[path[level - 1], i] 
            bound   = temp
            visited = [False] * len(visited) 
            for j in range(level): 
                if (path[j] != -1): 
                    visited[path[j]] = True
    return route, distance, bound, weight, path, visited




def branch_and_bound(distance_matrix): 
    distance   = float('+inf')
    path       = [  -1   ] * (distance_matrix.shape[0] + 1) 
    path[0]    = 0
    visited    = [ False ] *  distance_matrix.shape[0]
    visited[0] = True
    route      = [ None  ] * (distance_matrix.shape[0] + 1)
    weight     = 0
    level      = 1
    bound      = np.ceil(sum([ (min_1(distance_matrix, i) + min_2(distance_matrix, i)) for i in range(0, distance_matrix.shape[0])])/2) 
    route, distance, bound, weight, path, visited = explore_path(route, distance, distance_matrix, bound, weight, level, path, visited) 
    route      = [item+1 for item in route]
    return route, distance



O algoritmo abaixo foi o que eu desenvolvi

In [ ]:
import numpy as np

def branch_and_bound_recursive(graph, current_bound, current_weight, level, current_path, closest, visited, current_result):
    if level == graph.number_of_nodes():
        if current_path[level - 1] != current_path[0]:
            result = current_weight + graph[current_path[level-1]][current_path[0]]['weight']
            if result < current_result:
                current_result = result
        return current_result

    for next_node in graph.nodes():
        if current_path[level-1] != next_node and not visited[next_node]:
            temp_bound = current_bound
            current_weight += graph[current_path[level-1]][next_node]['weight']

            if level == 1:
                current_bound -= ((closest[current_path[level-1]][0] + closest[next_node][0]) / 2)
            else:
                current_bound -= ((closest[current_path[level-1]][1] + closest[next_node][0]) / 2)

            if (current_bound + current_weight) < current_result:
                current_path[level], visited[next_node] = next_node, True
                current_result = bnb_recursive(graph, current_bound, current_weight, level+1, current_path, closest, visited, current_result)

            current_weight -= graph[current_path[level-1]][next_node]['weight']
            current_bound = temp_bound

            visited = [False for _ in graph.nodes()]
            for i in range(level):
                visited[current_path[i]] = True

    return current_result

def branch_and_bound(graph):
    current_bound, closest = 0, []
    current_path, visited = [-1] * (graph.number_of_nodes() + 1), [False] * graph.number_of_nodes()
    visited[0], current_path[0] = True, 0

    for node in graph.nodes():
        min_weight_neighbors = sorted(graph[node].items(), key=lambda e: e[1]["weight"] if e[1]["weight"] != 0 else 1000000000)[:2]
        closest.append([min_weight_neighbors[0][1]['weight'], min_weight_neighbors[1][1]['weight']])
        current_bound += (min_weight_neighbors[0][1]['weight'] + min_weight_neighbors[1][1]['weight']) / 2

    return bnb_recursive(graph, current_bound, 0, 1, current_path, closest, visited, np.Inf)


### Algoritmo Twice around the tree

In [ ]:
import copy
from scipy.sparse.csgraph import minimum_spanning_tree

def distance_calc(distance_matrix, city_tour):
    distance = 0
    for k in range(0, len(city_tour[0])-1):
        m = k + 1
        distance = distance + distance_matrix[city_tour[0][k]-1, city_tour[0][m]-1]            
    return distance

def calculate_tsp_route(distance_matrix, verbose=True):
    
    graph_T = minimum_spanning_tree(distance_matrix)
    graph_T = graph_T.toarray().astype(int)
    graph_H = np.zeros((graph_T.shape))
    for i in range(0, graph_T.shape[0]):
        for j in range(0, graph_T.shape[1]):
            if graph_T[i, j] > 0:
                graph_H[i, j] = 1
                graph_H[j, i] = 1
    H = nx.from_numpy_matrix(graph_H)
    if nx.is_eulerian(H):
        euler = list(nx.eulerian_path(H))
    else:
        H = nx.eulerize(H)
        euler = list(nx.eulerian_path(H))
    route = []
    for path in euler:
        i, j = path
        if i not in route:
            route.append(i)
        if j not in route:
            route.append(j)
    route = route + [route[0]]
    route = [item + 1 for item in route]
    
    
    return route

def calculate_tsp_distance(distance_matrix, tsp_route):
    return distance_calc(distance_matrix, [tsp_route, 1])






### Christofides

In [ ]:

from random import randint

def christofides_tsp(graph, timeout = 2):
    mst = minimal_spanning_tree(graph, 'Prim', starting_node=0)
    odd_degree_nodes = list(_get_odd_degree_vertices(mst))
    odd_degree_nodes_ix = np.ix_(odd_degree_nodes, odd_degree_nodes)
    nx_graph = nx.from_numpy_array(-1 * graph[odd_degree_nodes_ix])
    matching = max_weight_matching(nx_graph, maxcardinality=True)
    euler_multigraph = nx.MultiGraph(mst)
    for edge in matching:
        euler_multigraph.add_edge(odd_degree_nodes[edge[0]], odd_degree_nodes[edge[1]],
                                  weight=graph[odd_degree_nodes[edge[0]]][odd_degree_nodes[edge[1]]])
    euler_tour = list(eulerian_circuit(euler_multigraph, source=0))
    path = list(itertools.chain.from_iterable(euler_tour))
    
    return _remove_repeated_vertices(path, 0)[:-1]

def _get_odd_degree_vertices(graph):
    odd_degree_vertices = set()
    for index, row in enumerate(graph):
        if len(np.nonzero(row)[0]) % 2 != 0:
            odd_degree_vertices.add(index)
    return odd_degree_vertices

def _remove_repeated_vertices(path, starting_node):
    path = list(dict.fromkeys(path).keys())
    path.append(starting_node)
    return path

def minimal_spanning_tree(graph, mode='Prim', starting_node=None):
    if mode == 'Prim':
        return _minimal_spanning_tree_prim(graph, starting_node)

def _minimal_spanning_tree_prim(graph, starting_node):
    node_count = len(graph)
    all_nodes = [i for i in range(node_count)]

    if starting_node is None:
        starting_node = randint(0, node_count-1)

    unvisited_nodes = all_nodes
    visited_nodes = [starting_node]
    unvisited_nodes.remove(starting_node)
    mst = np.zeros((node_count, node_count))

    while len(visited_nodes) != node_count:
        selected_subgraph = graph[np.array(visited_nodes)[:, None], np.array(unvisited_nodes)]
        min_edge_index = np.unravel_index(np.ma.masked_equal(selected_subgraph, 0, copy=False).argmin(),
                                          selected_subgraph.shape)
        edge_from = visited_nodes[min_edge_index[0]]
        edge_to = unvisited_nodes[min_edge_index[1]]
        mst[edge_from, edge_to] = graph[edge_from, edge_to]
        mst[edge_to, edge_from] = graph[edge_from, edge_to]
        unvisited_nodes.remove(edge_to)
        visited_nodes.append(edge_to)
    return mst

def route_cost(graph, path):
    cost = 0
    for index in range(len(path) - 1):
        cost = cost + graph[path[index]][path[index + 1]]
    cost = cost + graph[path[-1], path[0]]
    return cost

### TESTES

In [ ]:
import os
import time
import psutil
import pandas as pd
import itertools


def get_memory_usage():
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss  
    return memory_usage / (1024 ** 2)  


folder_path = 'C:/Users/bruno/Downloads/TSP'
results = []

for filename in os.listdir(folder_path):
    if filename.endswith(".tsp"):
        file_path = os.path.join(folder_path, filename)

    
        coordinates = read_tsp_file(file_path)
        dist_matrix = distance_matrix(coordinates, coordinates)

       
        start_time_tat = time.time()
        tsp_route_twice_around = calculate_tsp_route(dist_matrix, verbose=False)
        tsp_distance_twice_around = calculate_tsp_distance(dist_matrix, tsp_route_twice_around)
        end_time_tat = time.time()
        elapsed_time_tat = end_time_tat - start_time_tat

        
        start_time_christofides = time.time()
        tsp_path_christofides = christofides_tsp(dist_matrix)
        tsp_cost_christofides = route_cost(dist_matrix, tsp_path_christofides)
        end_time_christofides = time.time()
        elapsed_time_christofides = end_time_christofides - start_time_christofides

      
        memory_usage = get_memory_usage()
        print(filename, tsp_distance_twice_around, tsp_cost_christofides,
              elapsed_time_tat, elapsed_time_christofides, memory_usage)
            
        results.append({
            'Instance': filename,
            'TSP Twice Around Cost': tsp_distance_twice_around,
            'Elapsed Time (TAT)': elapsed_time_tat,
            'Memory Usage (TAT) (MB)': memory_usage,
            'TSP Christofides Cost': tsp_cost_christofides,
            'Elapsed Time (Christofides)': elapsed_time_christofides,
            'Memory Usage (Christofides) (MB)': memory_usage
        })
     
        
results_df = pd.DataFrame(results)
results_df.to_csv('results.csv', index=False)

print("Results saved to 'results.csv'")
